In [64]:
import os
from dotenv import load_dotenv
load_dotenv()


service_endpoint = os.getenv("AZURE_SEARCH_SERVICE_ENDPOINT")
key = os.getenv("AZURE_SEARCH_API_KEY")
connection_string = os.getenv("AZURE_STORAGE_CONNECTION_STRING")
cosmos_db_connection_string = os.getenv("AZURE_COSMOS_DB_CONN_STRING")

from azure.core.credentials import AzureKeyCredential
from azure.search.documents.indexes.models import (
    SearchIndexerDataContainer,
    SearchIndexerDataSourceConnection,
    SearchIndex,
    SearchIndexer,
    SimpleField,
    SearchFieldDataType,
)
from azure.search.documents.indexes import SearchIndexClient, SearchIndexerClient

indexers_client = SearchIndexerClient(service_endpoint, AzureKeyCredential(key))

In [66]:
service_endpoint = os.getenv('AZURE_SEARCH_SERVICE_ENDPOINT')
search_api_key = os.getenv('AZURE_SEARCH_API_KEY')
cosmos_db_account_endpoint = os.getenv('COSMOS_DB_ACCOUNT_ENDPOINT')
cosmos_db_account_key = os.getenv('COSMOS_DB_ACCOUNT_KEY')
cosmos_db_database_name = "nome-do-seu-database"
cosmos_db_container_name = 'nome-do-seu-container'



### INDEXER depende de um DATASOURCE e um INDEX, então :
### cria um index, um data source e um indexer 

In [ ]:
from azure.search.documents.indexes.models import (
    SearchIndex, SimpleField, SearchFieldDataType, SearchableField, SearchIndexerDataContainer, 
    SearchIndexerDataSourceConnection, SearchIndexer
)
from azure.core.credentials import AzureKeyCredential
from azure.search.documents.indexes import SearchIndexClient
from azure.search.documents.indexes import SearchIndexerClient
import os
from dotenv import load_dotenv

# Carregar variáveis de ambiente do arquivo .env
load_dotenv()

# Obter as variáveis de ambiente
service_endpoint = os.getenv('AZURE_SEARCH_SERVICE_ENDPOINT')
search_api_key = os.getenv('AZURE_SEARCH_API_KEY')
cosmos_db_database_name = "DB-1"
cosmos_db_container_name = 'container-1'
cosmos_db_account_endpoint = os.getenv('COSMOS_DB_ACCOUNT_ENDPOINT')
cosmos_db_account_key = os.getenv('COSMOS_DB_ACCOUNT_KEY')

# Definir a string de conexão do Cosmos DB
cosmos_db_connection_string = f"AccountEndpoint={cosmos_db_account_endpoint};AccountKey={cosmos_db_account_key};Database={cosmos_db_database_name}"

def create_indexer():
    # Nome do índice
    index_name = "index-viacod-1"
    # index_name = "index-docs-10072024"

    # Definir os campos do índice de acordo com a estrutura dos documentos JSON
    fields = [
        SimpleField(name="id", type=SearchFieldDataType.String, key=True),
        SearchableField(name="documento", type=SearchFieldDataType.String, filterable=True),
        SimpleField(name="totalPaginas", type=SearchFieldDataType.Int32, filterable=True),
        SearchableField(name="texto", type=SearchFieldDataType.String),
        SimpleField(name="pagina", type=SearchFieldDataType.Int32, filterable=True),
        SimpleField(name="conteudoVector", type=SearchFieldDataType.Collection(SearchFieldDataType.Double), searchable=True),
        SimpleField(name="embeddingModel", type=SearchFieldDataType.String)
    ]

    # Criar o índice
    index = SearchIndex(name=index_name, fields=fields)
    ind_client = SearchIndexClient(service_endpoint, AzureKeyCredential(search_api_key))
    ind_client.create_or_update_index(index)
    print(f"Índice '{index_name}' criado com sucesso.")

    name_ds = "indexer-datasource-1"
    # Criar um datasource para Cosmos DB
    container = SearchIndexerDataContainer(name=cosmos_db_container_name)
    data_source_connection = SearchIndexerDataSourceConnection(
        name=name_ds,
        type="cosmosdb",
        connection_string=cosmos_db_connection_string,
        container=container
    )
    indexers_client = SearchIndexerClient(service_endpoint, AzureKeyCredential(search_api_key))
    try:
        data_source = indexers_client.create_data_source_connection(data_source_connection)
        print(f"Datasource {name_ds} criado com sucesso.")
    except Exception as e:
        if "already exists" in str(e):
            data_source = indexers_client.get_data_source_connection(name_ds)
            indexers_client.create_or_update_data_source_connection(data_source_connection)
            print(f"Datasource {name_ds} atualizado com sucesso.")
        else:
            raise e

    # Criar um indexador
    indexer_name = "indexer-viacod-1" 
    indexer = SearchIndexer(
        name=indexer_name,
        data_source_name=name_ds,
        target_index_name=index_name
    )
    result = indexers_client.create_or_update_indexer(indexer)
    print(f"Indexer {indexer_name} criado com sucesso.")

create_indexer()


### cria apenas indexer, considerando que já há index e data source criados 

In [ ]:

def create_indexer():
    # Nome do índice
    index_name = "index-via-cod-c-pesquisa-semantica-1"

    name_ds = "indexer-datasource-10072024"
    # Criar um datasource para Cosmos DB

    # Criar um indexador
    indexer_name = "indexer-viacod-c-pesquisa-semantica-22082024-1232" 
    indexer = SearchIndexer(
        name=indexer_name,
        data_source_name=name_ds,
        target_index_name=index_name
    )
    result = indexers_client.create_or_update_indexer(indexer)
    print(f"Indexer {indexer_name} criado com sucesso.")

create_indexer()


In [ ]:
def list_indexers():
    result = indexers_client.get_indexers()
    names = [x.name for x in result]
    print("Found {} Indexers in the service: \n {}".format(len(result), ", ".join(names)))

list_indexers()

In [8]:
indexer_name = 'my-indexer-name'
def get_indexer():
    result = indexers_client.get_indexer(indexer_name)
    print("Retrived Indexer: ")
    return result


In [ ]:
get_indexer()

In [ ]:
def get_indexer_status():
    # [START get_indexer_status]
    result = indexers_client.get_indexer_status("indexer-viacod-c-pesquisa-semantica-22082024-1232")
    print("Retrived Indexer status :")
    return result
    # [END get_indexer_status]
get_indexer_status()


In [ ]:
indexer_name_to_run = 'nome do indexer que deseja rodar'

def run_indexer():
    result = indexers_client.run_indexer(indexer_name_to_run)
    print(f"Ran the Indexer {indexer_name_to_run}")
    return result
run_indexer()

In [ ]:
indexer_name_to_reset = 'nome do indexer que deseja resetar'
def reset_indexer():
    result = indexers_client.reset_indexer(indexer_name_to_reset)
    print(f"Indexer {indexer_name_to_reset} has been reset")
    return result

In [ ]:
from azure.core.exceptions import ResourceNotFoundError

def delete_indexer(indexer_name):
    try:
        # Verificar se o indexador existe antes de tentar excluí-lo
        indexer = indexers_client.get_indexer(indexer_name)
        if indexer:
            indexers_client.delete_indexer(indexer_name)
            print(f"Indexer '{indexer_name}' successfully deleted.")
    except ResourceNotFoundError:
        print(f"Indexer '{indexer_name}' does not exist.")
    except Exception as e:
        print(f"Failed to delete indexer '{indexer_name}'. Error: {e}")

# Solicita o nome do indexador ao usuário e passa para a função
delete_indexer(input('Digite o nome do indexer que deseja excluir: '))
